In [1]:
import pandas as pd
import os
import glob
import chardet
import re

In [2]:
# Specify the folder path
folder_path = '/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/census_raw'

# Construct file paths for the CSV files
ref_codes_df_path = os.path.join(folder_path, 'county_fips_master.csv')
counties_2001_2010_path = os.path.join(folder_path, 'co-estint-tot_2001_2010.csv')
counties_2010_2020_path = os.path.join(folder_path, 'co-est2010_2020.csv')
counties_2020_2022_path = os.path.join(folder_path, 'co-est2020_2022-alldata.csv')

# Read CSV files into DataFrames with the correct encoding
ref_codes_df = pd.read_csv(ref_codes_df_path, encoding='latin-1')  # Use 'latin-1' encoding
cntys_2001_2010 = pd.read_csv(counties_2001_2010_path, encoding='latin-1')  # Use 'latin-1' encoding
cntys_2010_2020 = pd.read_csv(counties_2010_2020_path, encoding='latin-1')  # Use 'latin-1' encoding
cntys_2020_2022 = pd.read_csv(counties_2020_2022_path, encoding='latin-1')  # Use 'latin-1' encoding


In [3]:
# drop the overlapping data from the older datasets
cntys_2001_2010 = cntys_2001_2010.drop('POPESTIMATE2010', axis=1)
cntys_2010_2020 = cntys_2010_2020.drop('POPESTIMATE2020', axis=1)

#and drop the extra pop data; the BEA data only goes through 2001-2021
cntys_2001_2010 = cntys_2001_2010.drop('POPESTIMATE2000', axis=1)
cntys_2020_2022 = cntys_2020_2022.drop('POPESTIMATE2022', axis=1)

In [4]:
# show that the Doña Ana County in the '20-'22 dataset is encoded incorrectly
print(cntys_2020_2022[(cntys_2020_2022["COUNTY"] == 13) & (cntys_2020_2022['STNAME'] == 'New Mexico')])

# and fix it!
# Find the specific row
mask = (cntys_2020_2022["COUNTY"] == 13) & (cntys_2020_2022['STNAME'] == 'New Mexico')

# Rename the 'CTYNAME' for the specific row to "Doña Ana County"
cntys_2020_2022.loc[mask, 'CTYNAME'] = 'Doña Ana County'

# now double check
print(cntys_2020_2022[(cntys_2020_2022["COUNTY"] == 13) & (cntys_2020_2022['STNAME'] == 'New Mexico')])

      SUMLEV  REGION  DIVISION  STATE  COUNTY      STNAME           CTYNAME  \
1836      50       4         8     35      13  New Mexico  DoÃ±a Ana County   

      ESTIMATESBASE2020  POPESTIMATE2020  POPESTIMATE2021  ...  RDEATH2021  \
1836             219567           220047           221655  ...   10.731217   

      RDEATH2022  RNATURALCHG2021  RNATURALCHG2022  RINTERNATIONALMIG2021  \
1836   11.038401         0.117726        -0.471919               1.371966   

      RINTERNATIONALMIG2022  RDOMESTICMIG2021  RDOMESTICMIG2022  RNETMIG2021  \
1836               3.761865          5.293161          4.530419     6.665127   

      RNETMIG2022  
1836     8.292284  

[1 rows x 50 columns]
      SUMLEV  REGION  DIVISION  STATE  COUNTY      STNAME          CTYNAME  \
1836      50       4         8     35      13  New Mexico  Doña Ana County   

      ESTIMATESBASE2020  POPESTIMATE2020  POPESTIMATE2021  ...  RDEATH2021  \
1836             219567           220047           221655  ...   10.73

In [5]:
# Merge the three DataFrames based on the common columns: region, division, state, and county
firstmerge_df = cntys_2001_2010.merge(cntys_2010_2020, on=['REGION', 'DIVISION', 'STATE',
                                                           'COUNTY', 'STNAME', 'CTYNAME'], how='inner')
merged_df = firstmerge_df.merge(cntys_2020_2022, on=['REGION', 'DIVISION', 'STATE', 
                                                     'COUNTY', 'STNAME', 'CTYNAME'], how='inner')

In [6]:
# b) Change all column names to lowercase
merged_df.columns = merged_df.columns.str.lower()

# Merge based on common columns (region, division, state, county)
merged_df = merged_df.merge(ref_codes_df[['region', 'division', 'state', 'county', 'fips']], 
                                  on=['region', 'division', 'state', 'county'], how='left')

merged_df.reset_index(drop=True, inplace=True)

In [7]:
# Create a regular expression pattern for "popestimate" columns
pattern = re.compile(r'^popestimate\d{4}$')

# Keep only the specified columns and columns that match the "popestimate" pattern
columns_to_keep = ['region', 'division', 'state', 'county', 'stname', 'ctyname', 'fips'] + [col for col in merged_df.columns if pattern.match(col)]

# Select only the specified columns
merged_df = merged_df[columns_to_keep]

In [8]:
# List of full state names
selected_states = ['Arizona', 'California', 'Colorado', 'Idaho', 'Montana', 'New Mexico', 'Nevada', 'Oregon', 'Utah', 'Washington', 'Wyoming']

# Filter the DataFrame to include only the selected states
filtered_df = merged_df[merged_df['stname'].isin(selected_states)].copy()

# Filter out rows where 'ctyname' is the same as 'stname' so we only work with counties, not the state-level rows
filtered_df = filtered_df[filtered_df['ctyname'] != filtered_df['stname']]

# convert FIPS to integer and change name to align with BEA nomenclature
filtered_df['fips'] = filtered_df['fips'].astype(int)
filtered_df = filtered_df.rename(columns={'fips': 'geofips'})

In [9]:
# Create a dictionary to map old column names to new column names, to get the years labeled correctly
columns_to_rename = {}
for col in filtered_df.columns:
    if col.startswith('popestimate'):
        year = col[-4:]  # Extract the year from the column name
        new_col_name = f'yr_{year}'
        columns_to_rename[col] = new_col_name

# Rename the columns using the dictionary
filtered_df = filtered_df.rename(columns=columns_to_rename)

In [10]:
# Add two new cols to help with merging w BEA data

# Add the "Description" column with the value "population"
filtered_df['description'] = 'population'

# Add the "LineCode" column with the value 20
filtered_df['linecode'] = 20

In [11]:
# Get the unique state names from 'filtered_df'
unique_states = filtered_df['stname'].unique()
unique_cntys = filtered_df['geofips'].unique()


# Check if there are exactly 11 unique states
if (len(unique_states) == 11) and (len(unique_cntys) == 414):
    print("There are 11 unique states and 414 unique counties represented.")
else:
    print(f"Error: There are {len(unique_states)} unique states, but there should be 11.")

There are 11 unique states and 414 unique counties represented.


In [12]:
# Create the output file path using the state abbreviation
output_file_path = f'/Users/natebender/Documents/ArcGIS/Projects/Western_protected_lands/census_clean/census_2001_2021_clean.csv'

# Save the cleaned DataFrame to the output file
filtered_df.to_csv(output_file_path, index=False)